<a href="https://colab.research.google.com/github/hargurjeet/bt/blob/main/NY_taxi_fare_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Intitial Observations

- It is a supervised machine learning problem
- The dataset contians overall 50,000 records
- Following are the feature of the dataset
  - key - Unique string identifying each row in both the training and test sets. Comprised of pickup_datetime plus a unique integer, but this doesn't matter, it should just be used as a unique ID field. 
  - fare_amount (target)- float dollar amount of the cost of the taxi ride. 
  - pickup_datetime - timestamp value indicating when the taxi ride started.
  - pickup_longitude - float for longitude coordinate of where the taxi ride started.
  - pickup_latitude - float for latitude coordinate of where the taxi ride started.
  - dropoff_longitude - float for longitude coordinate of where the taxi ride ended.
  - dropoff_latitude - float for latitude coordinate of where the taxi ride ended.
  - passenger_count - integer indicating the number of passengers in the taxi ride.

In [121]:
## Perfoming the required imports
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression

In [122]:
required_cols = ['fare_amount', 'pickup_datetime', 'pickup_longitude',
       'pickup_latitude', 'dropoff_longitude', 'dropoff_latitude',
       'passenger_count']

dtypes = {
    'fare_amount' : 'float32', 
    'pickup_datetime': 'float32',
    'pickup_longitude': 'float32',
    'pickup_latitude': 'float32',
    'dropoff_longitude': 'float32',
    'dropoff_latitude': 'float32',
    'passenger_count': 'uint8'
}
file_path = 'https://raw.githubusercontent.com/hargurjeet/bt/main/ny_taxi_fare_data.csv'
df = pd.read_csv(file_path, 
                 usecols = required_cols, 
                 parse_dates=['pickup_datetime'],
                 dtype = dtypes)
df.head()

,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
0,4.5,2009-06-15 17:26:21+00:00,-73.844315,40.721317,-73.841614,40.712276,1
1,16.9,2010-01-05 16:52:16+00:00,-74.016045,40.711304,-73.979271,40.782005,1
2,5.7,2011-08-18 00:35:00+00:00,-73.982735,40.761269,-73.991241,40.750561,2
3,7.7,2012-04-21 04:30:42+00:00,-73.987129,40.733143,-73.991570,40.758091,1
4,5.3,2010-03-09 07:51:00+00:00,-73.968094,40.768009,-73.956657,40.783764,1


In [123]:
df.shape

(50000, 7)

# Data Exploration

- Basics info regarding dataset
- EDA and visualizaitons
- Key insights

In [124]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype              
---  ------             --------------  -----              
 0   fare_amount        50000 non-null  float32            
 1   pickup_datetime    50000 non-null  datetime64[ns, UTC]
 2   pickup_longitude   50000 non-null  float32            
 3   pickup_latitude    50000 non-null  float32            
 4   dropoff_longitude  50000 non-null  float32            
 5   dropoff_latitude   50000 non-null  float32            
 6   passenger_count    50000 non-null  uint8              
dtypes: datetime64[ns, UTC](1), float32(5), uint8(1)
memory usage: 1.4 MB


There seems to be no null values within the dataset

In [125]:
df.describe()

,fare_amount,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
count,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000
mean,11.364215,-72.521416,39.931904,-72.517723,39.924244,1.667840
std,9.685438,10.392804,6.224685,10.406597,6.014816,1.289195
min,-5.000000,-75.423851,-74.006889,-84.654243,-74.006378,0.000000
25%,6.000000,-73.992065,40.734879,-73.991150,40.734371,1.000000
50%,8.500000,-73.981842,40.752678,-73.980080,40.753372,1.000000
75%,12.500000,-73.967148,40.767361,-73.963585,40.768166,2.000000
max,200.000000,40.783470,401.083344,40.851028,43.415192,6.000000


In [126]:
def min_max_date(df, date_col):
  return df[date_col].max(), df[date_col].min()

min_max_date(df, 'pickup_datetime')

(Timestamp('2015-06-30 22:42:39+0000', tz='UTC'),
 Timestamp('2009-01-01 01:31:49+0000', tz='UTC'))

Obseravations - 
- min fare amount is -5 dollars  and max is 200 dollars
- 50% values are under 8.5$, 75% of rides cost less than 12.5$. This gives the basic understanding how good our models need to be.
- While making prediction, I would expect my prediciton to be under +_ 3$ range otherwise I am off by a lot.
- max passege count is 6 which is highly unlikely. Hence we might requrie some data cleanup.
- observing the min and max date its about 6 years of worth of data

# 3. EDA and Visualization

#4. Preparing the dataset

We will set aside 20% of our data as validation set for evaluation of our model

In [127]:
train_df, val_df = train_test_split(df, test_size=0.2, random_state=42)

In [128]:
len(train_df), len(val_df)

(40000, 10000)

In [129]:
## extract input and targets

train_df.columns

Index(['fare_amount', 'pickup_datetime', 'pickup_longitude', 'pickup_latitude',
       'dropoff_longitude', 'dropoff_latitude', 'passenger_count'],
      dtype='object')

In [130]:
input_cols = ['pickup_longitude', 'pickup_latitude','dropoff_longitude', 'dropoff_latitude', 'passenger_count']
target_cols = 'fare_amount'

In [131]:
train_inputs = train_df[input_cols]
train_targets = train_df[target_cols]
val_inputs = val_df[input_cols]
val_targets = val_df[target_cols]

# 5. Train baseline models

In [132]:
class regressor():
  def fit(self, inputs, targets):
    self.mean = targets.mean()

  def predicts(self, inputs):
    return np.full(inputs.shape[0], self.mean)

In [133]:
np.full([40000], 40000)

array([40000, 40000, 40000, ..., 40000, 40000, 40000])

In [134]:
mean_model = regressor()

In [135]:
mean_model.fit(train_inputs, train_targets)
print(f'Average fare for the taxi id {mean_model.mean}')

Average fare for the taxi id 11.376873970031738


In [136]:
mean_model.predicts(train_inputs)

array([11.37687397, 11.37687397, 11.37687397, ..., 11.37687397,
       11.37687397, 11.37687397])

In [137]:
train_targets

39087    10.000000
30893     4.000000
45278     6.900000
16398     7.700000
13653     4.500000
           ...    
11284     6.500000
44732     3.700000
38158    12.100000
860      12.100000
15795    57.330002
Name: fare_amount, Length: 40000, dtype: float32

In [138]:
mean_model.predicts(val_inputs)

array([11.37687397, 11.37687397, 11.37687397, ..., 11.37687397,
       11.37687397, 11.37687397])

In [139]:
val_targets

33553     7.300000
9427     33.299999
199       5.500000
12447     7.000000
39489     5.300000
           ...    
28567     9.300000
25079     5.500000
18707     6.500000
15200    30.500000
5857      6.900000
Name: fare_amount, Length: 10000, dtype: float32

In [140]:
def rmse(targets, predicitions):
  return mean_squared_error(targets, predicitions, squared = False)

rmse(train_targets, mean_model.predicts(train_inputs))

9.696218748570448

The average score is really bad. Any model we train should be better than this.

In [141]:
## Trying linear regression

linear_model = LinearRegression()
linear_model.fit(train_inputs, train_targets)
train_preds = linear_model.predict(train_inputs)
train_preds

array([11.819403, 11.820234, 11.687607, ..., 11.41951 , 11.284868,
       11.286995], dtype=float32)

In [142]:
train_targets

39087    10.000000
30893     4.000000
45278     6.900000
16398     7.700000
13653     4.500000
           ...    
11284     6.500000
44732     3.700000
38158    12.100000
860      12.100000
15795    57.330002
Name: fare_amount, Length: 40000, dtype: float32

In [143]:
rmse(train_targets, train_preds)

9.694377

In [144]:
val_preds = linear_model.predict(val_inputs)
rmse(val_targets, val_preds)

9.641011

- May the model not learning well from lat and long data
- We are not using pickup date time, fare can be seasonal and can depend on time of the day

# 6. Feature Engineering

- Extract part of the dates
- Remove outlier and invalid data
- Add distances btw pickup and drop
- Add distnace from landmark

## Extract Parts of Date
 - Year
 - Month
 - Day
 - Weekday
 - hour

In [145]:
def part_of_dates(df, column):
  df[column+ '_year']= df[column].dt.year
  df[column+ '_month']= df[column].dt.month
  df[column+ '_day']= df[column].dt.day
  df[column+ '_week']= df[column].dt.weekday
  df[column+ '_hour']= df[column].dt.hour

In [146]:
part_of_dates(train_df, 'pickup_datetime')
part_of_dates(val_df, 'pickup_datetime')

In [147]:
train_df.head()

,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,pickup_datetime_year,pickup_datetime_month,pickup_datetime_day,pickup_datetime_week,pickup_datetime_hour
39087,10.0,2013-07-27 17:04:00+00:00,-73.974335,40.791428,-73.979034,40.766365,5,2013,7,27,5,17
30893,4.0,2013-01-08 09:26:00+00:00,-73.973656,40.751633,-73.969948,40.756702,5,2013,1,8,1,9
45278,6.9,2012-03-17 16:45:00+00:00,-73.975266,40.752281,-73.995094,40.737499,4,2012,3,17,5,16
16398,7.7,2012-06-08 09:01:17+00:00,-73.983032,40.766785,-73.971947,40.789288,1,2012,6,8,4,9
13653,4.5,2015-06-22 17:30:49+00:00,-73.986717,40.771648,-73.982140,40.770699,1,2015,6,22,0,17


## Add distance between pickup and dropp

- https://en.wikipedia.org/wiki/Haversine_formula


In [148]:
def haversine_np(lon1, lat1, lon2, lat2):
    """
    Calculate the great circle distance between two points
    on the earth (specified in decimal degrees)

    All args must be of equal length.    

    """
    lon1, lat1, lon2, lat2 = map(np.radians, [lon1, lat1, lon2, lat2])

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = np.sin(dlat/2.0)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2.0)**2

    c = 2 * np.arcsin(np.sqrt(a))
    km = 6367 * c
    return km

In [149]:
def trip_distance(df):
  df['trip_distance'] = haversine_np(df['pickup_longitude'],
                                     df['pickup_latitude'],
                                     df['dropoff_longitude'],
                                     df['dropoff_latitude'])

In [150]:
trip_distance(train_df)
trip_distance(val_df)

In [151]:
train_df[((train_df.trip_distance >=8) | (train_df.trip_distance <= 4))]

,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,pickup_datetime_year,pickup_datetime_month,pickup_datetime_day,pickup_datetime_week,pickup_datetime_hour,trip_distance
39087,10.000000,2013-07-27 17:04:00+00:00,-73.974335,40.791428,-73.979034,40.766365,5,2013,7,27,5,17,2.813101
30893,4.000000,2013-01-08 09:26:00+00:00,-73.973656,40.751633,-73.969948,40.756702,5,2013,1,8,1,9,0.643929
45278,6.900000,2012-03-17 16:45:00+00:00,-73.975266,40.752281,-73.995094,40.737499,4,2012,3,17,5,16,2.341894
16398,7.700000,2012-06-08 09:01:17+00:00,-73.983032,40.766785,-73.971947,40.789288,1,2012,6,8,4,9,2.669228
13653,4.500000,2015-06-22 17:30:49+00:00,-73.986717,40.771648,-73.982140,40.770699,1,2015,6,22,0,17,0.399969
...,...,...,...,...,...,...,...,...,...,...,...,...,...
16850,7.500000,2014-12-05 18:00:56+00:00,-73.983727,40.766174,-73.977547,40.777889,1,2014,12,5,4,18,1.401790
11284,6.500000,2010-12-07 11:23:00+00:00,-73.980911,40.767860,-73.980209,40.780342,1,2010,12,7,1,11,1.388346
44732,3.700000,2009-06-30 22:39:25+00:00,-73.960564,40.775860,-73.961830,40.771255,1,2009,6,30,1,22,0.522993
38158,12.100000,2010-09-14 22:34:00+00:00,-73.974663,40.751743,-73.995987,40.744347,2,2010,9,14,1,22,1.974436


Distance from popular landmarks

- JFK airport
- LGA Airport
- EWR airport
- WTC

In [152]:
jfk_lonlat = -73.7781, 40.6413
lga_lonlat = -73.8740, 40.7769
ewr_lonlat = -74.1745, 40.6895
wtc_lonlat = -74.0099, 40.7126

## Remove outlier and invalid data

- Fare amount
- Passange count
- Pickup latitude and longitude
- drop latitude and longitude

In [153]:
df.describe()

,fare_amount,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
count,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000
mean,11.364215,-72.521416,39.931904,-72.517723,39.924244,1.667840
std,9.685438,10.392804,6.224685,10.406597,6.014816,1.289195
min,-5.000000,-75.423851,-74.006889,-84.654243,-74.006378,0.000000
25%,6.000000,-73.992065,40.734879,-73.991150,40.734371,1.000000
50%,8.500000,-73.981842,40.752678,-73.980080,40.753372,1.000000
75%,12.500000,-73.967148,40.767361,-73.963585,40.768166,2.000000
max,200.000000,40.783470,401.083344,40.851028,43.415192,6.000000


we will use the following
- longitudes: -75 to -72
- latitudes: 40 to 42

In [154]:
train_df.shape, val_df.shape

((40000, 13), (10000, 13))

In [117]:
def remove_outlier(df):
  return df[(
      (df.pickup_longitude>= -75) & (df.pickup_longitude<=-72) & (df.pickup_latitude>= 40) & (df.pickup_latitude <=42) & 
      (df.dropoff_latitude>=40) &(df.dropoff_latitude<=42)& (df.dropoff_longitude>=-75) &(df.dropoff_longitude<=-72) 

  )]

In [155]:
train_df = remove_outlier(train_df)
val_df = remove_outlier(val_df)

In [156]:
train_df.shape, val_df.shape

((39158, 13), (9795, 13))

(10000, 13)